# 1. Load the Data

In [1]:
import pandas as pd

# train
train_click = pd.read_csv("data/train_click_log.csv")
articles = pd.read_csv("data/articles.csv")
articles = articles.rename(columns={'article_id': 'click_article_id'})  #重命名，方便后续match
articles_emb = pd.read_csv("data/articles_emb.csv")

# test
test_click = pd.read_csv("data/testA_click_log.csv")

"""
train_click = train_click.append(test_click)
train_click = train_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))

train_click = train_click.sort_values('user_id')
train_click.reset_index(drop=True, inplace=True)
train_click.head()
"""

train_click = test_click
train_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


In [2]:
train_click.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518010 entries, 0 to 518009
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   user_id              518010 non-null  int64
 1   click_article_id     518010 non-null  int64
 2   click_timestamp      518010 non-null  int64
 3   click_environment    518010 non-null  int64
 4   click_deviceGroup    518010 non-null  int64
 5   click_os             518010 non-null  int64
 6   click_country        518010 non-null  int64
 7   click_region         518010 non-null  int64
 8   click_referrer_type  518010 non-null  int64
dtypes: int64(9)
memory usage: 35.6 MB


In [3]:
train_click_matrix_df = train_click[["user_id", "click_article_id"]]

In [4]:
train_click_matrix_df.head()

,user_id,click_article_id
0,249999,160974
1,249999,160417
2,249998,160974
3,249998,202557
4,249997,183665


In [5]:
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

# 2. Data Matrix

In [6]:
train_click_matrix_df["count"] = 1
train_click_matrix_df.head()

,user_id,click_article_id,count
0,249999,160974,1
1,249999,160417,1
2,249998,160974,1
3,249998,202557,1
4,249997,183665,1


In [7]:
train_click_matrix_temp = train_click_matrix_df.groupby(["user_id", "click_article_id"])["count"].sum()
train_click_matrix_temp

user_id  click_article_id
200000   191971              1
         194300              1
         195839              1
200001   175040              1
200002   70335               1
                            ..
249999   299152              1
         301015              1
         313431              1
         336221              1
         352983              1
Name: count, Length: 512475, dtype: int64

In [8]:
user_item_dict = dict(train_click_matrix_df.groupby(["user_id"])["click_article_id"].apply(set))

In [9]:
train_click_matrix = train_click_matrix_temp.unstack(level=-1).fillna(0)
train_click_matrix

click_article_id,137,143,291,540,722,755,860,906,911,1550,...,363932,363947,363952,363957,363962,363967,363976,364014,364017,364043
user_id,,,,,,,,,,,,,,,,,,,,,
200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_click_dict = train_click_matrix_temp.unstack(level=-1)

In [11]:
article_id_index = list(train_click_matrix.columns)
user_id_index = list(train_click_matrix.index)
article_id_index[:3], user_id_index[:3]

([137, 143, 291], [200000, 200001, 200002])

In [12]:
click_matrix = train_click_matrix.values
click_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 3. Sparse Matrix

In [13]:
from scipy.sparse import coo_matrix

matrix = coo_matrix(train_click_matrix.values)

print("Matrix Shape::", matrix.shape)
print("Matrix Columns::", matrix.col)
print("Matrix Rows::",matrix.row)

Matrix Shape:: (50000, 16330)
Matrix Columns:: [ 8532  8579  8745 ... 14263 15116 15765]
Matrix Rows:: [    0     0     0 ... 49999 49999 49999]


# 4. Matrix Factorization 

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [15]:
matrix.shape

(50000, 16330)

In [16]:
U, sigma, V = svds(matrix, k=10)

sigma = np.diag(sigma)
sigma.shape

(10, 10)

In [17]:
F = U.T
Q = np.dot(sigma, V).T
Q.shape, F.shape

((16330, 10), (10, 50000))

In [18]:
temp_matrix = np.array([[1.,2.,3.],[11.,13.,17.], [21.,23.,41.]])
# temp_matrix = coo_matrix(temp_matrix)
U, sigma, V = svds(temp_matrix, k=2)
sigma = np.diag(sigma)
np.dot(np.dot(U, sigma), V)

array([[ 1.50848594,  1.66072436,  2.93052418],
       [10.99775228, 13.00149974, 17.00030711],
       [20.96459153, 23.02362549, 41.00483796]])

In [77]:
U, V, sigma

(array([[ 0.02520827, -0.06488098],
        [-0.90735717, -0.42033738],
        [ 0.41960398, -0.90504527]]),
 array([[-0.37854744, -0.69298911,  0.61356983],
        [-0.41662535, -0.46437504, -0.7815236 ]]),
 array([[ 3.02217614,  0.        ],
        [ 0.        , 56.87254272]]))

# 5. Loss Function

In [19]:
Q.shape, F.shape

((16330, 10), (10, 50000))

In [20]:
from numpy.linalg import norm
import numpy as np

def error(matrix, F, Q, lambda1=0.02, lambda2=0.02):
    ratings = matrix.data
    rows = matrix.row
    cols = matrix.col
    e = 0 
    for ui in range(len(ratings)):
        rui = ratings[ui]
        j = rows[ui]
        i = cols[ui]
        if rui > 0:
            e = e + pow(rui - np.dot(Q[i,:], F[:,j]), 2) \
                  + lambda1 * (pow(norm(Q[i,:]), 2)) \
                  + lambda2 * (pow(norm(F[:,j]), 2))
    return e

In [114]:
error(matrix, F, Q)

13713809.095128594

# 6. Training (Stochastic Gradient Descent)

In [21]:
def SGD(matrix, Q=Q, F=F, lamda=0.02, steps=1000, learning_rate=0.0001, eps=1e-3):
    
    rmse_log = []
    
    n, p = matrix.shape
    Q = np.array(Q)
    F = np.array(F)
    
    rmse = np.sqrt(error(matrix, F, Q) / len(matrix.data))
    print("Initial RMSE: {:.4f}".format(rmse))
    rmse_log.append(rmse)
    
    for step in range(steps):
        for ui in range(len(matrix.data)):
            rui = ratings[ui]
            j = matrix.row[ui]
            i = matrix.col[ui]
            if rui > 0:
                eui = rui - np.dot(Q[i,:], F[:,j])
                
                Q[i,:] = Q[i,:] + learning_rate * 2 * (eui * F[:,j] - lamda * Q[i,:])
                F[:,j] = F[:,j] + learning_rate * 2 * (eui * Q[i,:] - lamda * F[:,j])

        rmse = np.sqrt(error(matrix, F, Q) / len(matrix.data))
        print("Epoch {} RMSE: {:.4f}".format(step + 1, rmse))
        rmse_log.append(rmse)
        
    print("Final RMSE: {:.4f}".format(rmse))
    return Q, F, rmse_log

In [ ]:
Q_star, F_star, rmse_log = SGD(matrix)

In [154]:
recommend_matrix = np.matmul(Q_star, F_star)

In [156]:
recommend_matrix = recommend_matrix.T

In [164]:
recommend_matrix_df = pd.DataFrame(recommend_matrix, columns=article_id_index, index=user_id_index)
# recommend_matrix_df["user_id"] = recommend_matrix_df.index + 200000
recommend_matrix_df.head()

,137,143,291,540,722,755,860,906,911,1550,...,363932,363947,363952,363957,363962,363967,363976,364014,364017,364043
200000,0.000083,0.000083,0.000054,0.000041,0.000065,0.000065,0.000066,0.000066,0.000029,0.000002,...,0.000061,0.001029,0.000986,0.000020,0.000120,0.001508,0.000217,0.000049,0.000170,0.000064
200001,0.000473,0.000473,0.001442,0.000104,0.000315,0.000315,0.000102,0.000102,0.000062,0.000011,...,0.000120,0.006831,0.000683,0.000122,0.000091,0.001513,0.001509,-0.000049,0.001057,0.000480
200002,0.000544,0.000544,0.000366,0.000460,0.000958,0.000958,0.001018,0.001018,0.000324,0.000023,...,0.001088,0.012327,0.015206,0.000354,0.001862,0.022892,0.002509,0.000856,0.002828,0.000752
200003,0.001301,0.001301,0.001558,0.000521,0.000945,0.000945,0.000768,0.000768,0.000383,0.000033,...,0.000750,0.017030,0.011958,0.000382,0.001311,0.020999,0.003303,0.000587,0.002367,0.000865
200004,0.001072,0.001072,0.001143,0.000711,0.001376,0.001376,0.001097,0.001097,0.000544,0.000048,...,0.001320,0.017784,0.018924,0.000359,0.001700,0.027257,0.003734,0.001010,0.003642,0.000558


# 7. Recommend

In [203]:
user_item_score_list = []

for user_id in tqdm(range(200000, 250000)):
    
    user = user_id
    
    items = []
    scores = []
    
    article_candidate = list(recommend_matrix_df.loc[user_id].sort_values(ascending=False).index)
    article_candidate_score = list(recommend_matrix_df.loc[user_id].sort_values(ascending=False))
    
    for index in range(len(article_candidate)):
        article = article_candidate[index]
        score = article_candidate_score[index]
        if article not in user_item_dict[user]:
            items.append(article)
        if len(items) >= 5:
            break
    user_item_score_list.append((user, items))

100%|██████████| 50000/50000 [08:47<00:00, 94.72it/s] 


# 8. Submit

In [207]:
submit_df = pd.DataFrame(user_item_score_list, columns=["user_id", "articles"])
submit_df.head()

,user_id,articles
0,200000,"[235870, 59057, 133160, 288320, 95716]"
1,200001,"[20691, 96210, 111043, 293513, 95716]"
2,200002,"[293301, 233717, 123909, 288320, 313431]"
3,200003,"[235870, 95716, 20691, 336220, 59057]"
4,200004,"[123909, 199197, 59057, 313431, 123289]"


In [209]:
user_item_recommend_dict = dict(user_item_score_list)

In [229]:
submit_df = pd.DataFrame.from_dict(user_item_recommend_dict, 
                                   orient='index') 

submit_df["user_id"] = submit_df.index
submit_df = submit_df[["user_id", 0 , 1, 2, 3 ,4]]
submit_df.head()

,user_id,0,1,2,3,4
200000,200000,235870,59057,133160,288320,95716
200001,200001,20691,96210,111043,293513,95716
200002,200002,293301,233717,123909,288320,313431
200003,200003,235870,95716,20691,336220,59057
200004,200004,123909,199197,59057,313431,123289


In [232]:
submit_df = submit_df.rename(columns={0:"article_1", 1:"article_2", 2:"article_3", 3:"article_4", 4:"article_5"})

In [233]:
submit_df

,user_id,article_1,article_2,article_3,article_4,article_5
200000,200000,235870,59057,133160,288320,95716
200001,200001,20691,96210,111043,293513,95716
200002,200002,293301,233717,123909,288320,313431
200003,200003,235870,95716,20691,336220,59057
200004,200004,123909,199197,59057,313431,123289
...,...,...,...,...,...,...
249995,249995,159762,235616,235870,293301,313431
249996,249996,16129,162300,300470,225463,202557
249997,249997,123757,20691,123289,128260,124748
249998,249998,293301,159762,233717,31836,284547


In [235]:
submit_df.to_csv("latent_factor_baseline.csv", index=False, header=True)